In [0]:
#@title
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# coding=utf-8
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression




############################ 加载数据 ############################
df_all = pd.read_csv("drive/My Drive/sougou/data/all_data.csv",encoding="UTF-8",index_col=0)
df_all.head() 

,ID,Age,Gender,Education,Query_List,Query_List1,jieba,jieba2
0,22DD920316420BE2DF8D6EE651BA174B,1.0,1.0,4.0,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,柔和双沟\t女生\t中财网首页 财经\t网址\t周公解梦大全查询2345\t曹云金再讽郭德纲...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再讽 ...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再...
1,43CC3AF5A8D6430A3B572337A889AFE4,2.0,1.0,3.0,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...","广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...",广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...,"广州 厨宝 烤箱 世情 薄 , 人情 恶 , 雨送 黄昏 花易落 , 晓 风干 , 泪痕 厦..."
2,6931EFC26D229CCFCEA125D3F3C21E57,4.0,2.0,3.0,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 壮观 衣服 网站 动漫 绘图 软件...,最 受欢迎 狗狗 排行榜 舶 怎么 读 场景 描 写 范例 三维 绘图 软件 枣 和 酸...
3,E780470C3BB0D340334BD08CDCC3C71A,2.0,2.0,4.0,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 怎么回事 麦当劳 旋风 勺子 吉林市 鹿...,干槽症 能 自愈 吗 太太 万岁 叶 舒心 去 没 去 美国 干槽症 右 眼皮 下面 一直 ...
4,0D58894CD481D4B3039000661665C0DB,3.0,2.0,3.0,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅 饲育 指南 韩姨 贵阳 古琴 洛基 风速 英美 店主 好吃 重生 儿童 寻家记 锦里 ...,貔貅 饲育 指南 韩姨 贵阳 学 古琴 洛基 风速 狗 综 英美 ] 店主 不 好吃 > 重...


In [0]:
df_all['Age'] = df_all['Age'] - 1
df_all['Gender'] = df_all['Gender'] - 1
df_all['Education'] = df_all['Education'] - 1
df_all.head()

,ID,Age,Gender,Education,Query_List,Query_List1,jieba,jieba2
0,22DD920316420BE2DF8D6EE651BA174B,0.0,0.0,3.0,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...,柔和双沟\t女生\t中财网首页 财经\t网址\t周公解梦大全查询2345\t曹云金再讽郭德纲...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再讽 ...,柔和 双沟 女生 中财网 首页 财经 网址 周公 解梦 大全 查询 2345 曹云金 再...
1,43CC3AF5A8D6430A3B572337A889AFE4,1.0,0.0,2.0,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...","广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只...",广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...,"广州 厨宝 烤箱 世情 薄 , 人情 恶 , 雨送 黄昏 花易落 , 晓 风干 , 泪痕 厦..."
2,6931EFC26D229CCFCEA125D3F3C21E57,3.0,1.0,2.0,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 壮观 衣服 网站 动漫 绘图 软件...,最 受欢迎 狗狗 排行榜 舶 怎么 读 场景 描 写 范例 三维 绘图 软件 枣 和 酸...
3,E780470C3BB0D340334BD08CDCC3C71A,1.0,1.0,3.0,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 怎么回事 麦当劳 旋风 勺子 吉林市 鹿...,干槽症 能 自愈 吗 太太 万岁 叶 舒心 去 没 去 美国 干槽症 右 眼皮 下面 一直 ...
4,0D58894CD481D4B3039000661665C0DB,2.0,1.0,2.0,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅饲育指南\t韩姨\t贵阳学古琴\t洛基\t风速狗\t综英美]店主不好吃\t>重生之被拐儿...,貔貅 饲育 指南 韩姨 贵阳 古琴 洛基 风速 英美 店主 好吃 重生 儿童 寻家记 锦里 ...,貔貅 饲育 指南 韩姨 贵阳 学 古琴 洛基 风速 狗 综 英美 ] 店主 不 好吃 > 重...


In [0]:
############################ tfidf ############################
tfv = TfidfVectorizer(min_df=3, max_df=0.95, sublinear_tf=True)
x_sp = tfv.fit_transform(df_all['jieba2'])

In [0]:
import time
def log(stri):
  now = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
  print(str(now) + ' ' + str(stri))

In [0]:
############################  Gender ############################
############################ lr stack ############################
tr_num = 88492
num_class = 2
n = 5

x = x_sp[:tr_num]
y = df_all['Gender'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Gender'][tr_num:]

stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    clf = LogisticRegression(C=2)
    clf.fit(x[tr], y[tr])
    y_pred_va = clf.predict_proba(x[va])
    y_pred_te = clf.predict_proba(x_te)
    log('va acc:%f' % clf.score(x[va],y[va]))
#     util.log('te acc:%f' % micro_avg_f1(y_te, clf.predict(x_te)))
    score_va += clf.score(x[va],y[va])
#     score_te += micro_avg_f1(y_te, clf.predict(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# util.log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
df_stack = pd.DataFrame(index=range(len(df_all)))
for i in range(stack_all.shape[1]):
    df_stack['tfidf_lr_age_{}'.format(i+1)] = stack_all[:, i]

df_stack.to_csv("drive/My Drive/sougou/data/lr_prob_gender.csv", index=None, encoding='utf8')

2020-04-10 18:21:23 stack:1/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:21:46 va acc:0.841234
2020-04-10 18:21:46 stack:2/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:22:08 va acc:0.841629
2020-04-10 18:22:08 stack:3/5


...

2020-04-10 18:22:31 va acc:0.840773
2020-04-10 18:22:31 stack:4/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:22:53 va acc:0.840208
2020-04-10 18:22:53 stack:5/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:23:15 va acc:0.846197
2020-04-10 18:23:15 va avg acc:0.842008


In [0]:
############################  Age ############################
############################ lr stack ############################
tr_num = 88492
num_class = 6
n = 5

x = x_sp[:tr_num]
y = df_all['Age'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Age'][tr_num:]

stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    clf = LogisticRegression(C=2)
    clf.fit(x[tr], y[tr])
    y_pred_va = clf.predict_proba(x[va])
    y_pred_te = clf.predict_proba(x_te)
    log('va acc:%f' % clf.score(x[va],y[va]))
#     util.log('te acc:%f' % micro_avg_f1(y_te, clf.predict(x_te)))
    score_va += clf.score(x[va],y[va])
#     score_te += micro_avg_f1(y_te, clf.predict(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# util.log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
df_stack = pd.DataFrame(index=range(len(df_all)))
for i in range(stack_all.shape[1]):
    df_stack['tfidf_lr_age_{}'.format(i+1)] = stack_all[:, i]

df_stack.to_csv("drive/My Drive/sougou/data/lr_prob_age.csv", index=None, encoding='utf8')

2020-04-10 17:57:36 stack:1/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 17:58:56 va acc:0.610486
2020-04-10 17:58:56 stack:2/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:00:17 va acc:0.607718
2020-04-10 18:00:17 stack:3/5


...

2020-04-10 18:01:39 va acc:0.611199
2020-04-10 18:01:39 stack:4/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:03:01 va acc:0.610690
2020-04-10 18:03:01 stack:5/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:04:23 va acc:0.611256
2020-04-10 18:04:23 va avg acc:0.610270


In [0]:
############################  Education ############################
############################ lr stack ############################
tr_num = 88492
num_class = 6
n = 5

x = x_sp[:tr_num]
y = df_all['Education'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Education'][tr_num:]

stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    clf = LogisticRegression(C=2)
    clf.fit(x[tr], y[tr])
    y_pred_va = clf.predict_proba(x[va])
    y_pred_te = clf.predict_proba(x_te)
    log('va acc:%f' % clf.score(x[va],y[va]))
#     util.log('te acc:%f' % micro_avg_f1(y_te, clf.predict(x_te)))
    score_va += clf.score(x[va],y[va])
#     score_te += micro_avg_f1(y_te, clf.predict(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# util.log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
df_stack = pd.DataFrame(index=range(len(df_all)))
for i in range(stack_all.shape[1]):
    df_stack['tfidf_lr_education_{}'.format(i+1)] = stack_all[:, i]

df_stack.to_csv("drive/My Drive/sougou/data/lr_prob_education.csv", index=None, encoding='utf8')

2020-04-10 18:23:17 stack:1/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:24:38 va acc:0.627041
2020-04-10 18:24:38 stack:2/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:26:02 va acc:0.630092
2020-04-10 18:26:02 stack:3/5


...

2020-04-10 18:27:25 va acc:0.623856
2020-04-10 18:27:25 stack:4/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:28:48 va acc:0.622217
2020-04-10 18:28:48 stack:5/5


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


2020-04-10 18:30:10 va acc:0.636061
2020-04-10 18:30:10 va avg acc:0.627853


In [0]:
from sklearn.naive_bayes import BernoulliNB

############################  Gender ############################
############################ bnb stack ############################
tr_num = 88492
num_class = 2
n = 5

x = x_sp[:tr_num]
y = df_all['Gender'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Gender'][tr_num:]

stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    clf = BernoulliNB()
    clf.fit(x[tr], y[tr])
    y_pred_va = clf.predict_proba(x[va])
    y_pred_te = clf.predict_proba(x_te)
    log('va acc:%f' % clf.score(x[va],y[va]))
#     util.log('te acc:%f' % micro_avg_f1(y_te, clf.predict(x_te)))
    score_va += clf.score(x[va],y[va])
#     score_te += micro_avg_f1(y_te, clf.predict(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# util.log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
df_stack = pd.DataFrame(index=range(len(df_all)))
for i in range(stack_all.shape[1]):
    df_stack['tfidf_bnb_gender_{}'.format(i+1)] = stack_all[:, i]

df_stack.to_csv("drive/My Drive/sougou/data/bnb_prob_gender.csv", index=None, encoding='utf8')

2020-04-10 18:14:01 stack:1/5
2020-04-10 18:14:02 va acc:0.834171
2020-04-10 18:14:02 stack:2/5
2020-04-10 18:14:03 va acc:0.832250
2020-04-10 18:14:03 stack:3/5
2020-04-10 18:14:04 va acc:0.834671
2020-04-10 18:14:04 stack:4/5
2020-04-10 18:14:05 va acc:0.835179
2020-04-10 18:14:05 stack:5/5
2020-04-10 18:14:05 va acc:0.832071
2020-04-10 18:14:06 va avg acc:0.833669


In [0]:
############################  Age ############################
############################ bnb stack ############################
tr_num = 88492
num_class = 6
n = 5

x = x_sp[:tr_num]
y = df_all['Age'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Age'][tr_num:]

stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    clf = BernoulliNB()
    clf.fit(x[tr], y[tr])
    y_pred_va = clf.predict_proba(x[va])
    y_pred_te = clf.predict_proba(x_te)
    log('va acc:%f' % clf.score(x[va],y[va]))
#     util.log('te acc:%f' % micro_avg_f1(y_te, clf.predict(x_te)))
    score_va += clf.score(x[va],y[va])
#     score_te += micro_avg_f1(y_te, clf.predict(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# util.log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
df_stack = pd.DataFrame(index=range(len(df_all)))
for i in range(stack_all.shape[1]):
    df_stack['tfidf_bnb_age_{}'.format(i+1)] = stack_all[:, i]

df_stack.to_csv("drive/My Drive/sougou/data/bnb_prob_age.csv", index=None, encoding='utf8')

2020-04-10 18:15:42 stack:1/5
2020-04-10 18:15:43 va acc:0.570767
2020-04-10 18:15:44 stack:2/5
2020-04-10 18:15:45 va acc:0.573592
2020-04-10 18:15:45 stack:3/5
2020-04-10 18:15:46 va acc:0.567804
2020-04-10 18:15:46 stack:4/5
2020-04-10 18:15:48 va acc:0.569273
2020-04-10 18:15:48 stack:5/5
2020-04-10 18:15:49 va acc:0.572381
2020-04-10 18:15:49 va avg acc:0.570763


In [0]:
############################  Education ############################
############################ bnb stack ############################
tr_num = 88492
num_class = 6
n = 5

x = x_sp[:tr_num]
y = df_all['Education'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Education'][tr_num:]

stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    clf = BernoulliNB()
    clf.fit(x[tr], y[tr])
    y_pred_va = clf.predict_proba(x[va])
    y_pred_te = clf.predict_proba(x_te)
    log('va acc:%f' % clf.score(x[va],y[va]))
#     util.log('te acc:%f' % micro_avg_f1(y_te, clf.predict(x_te)))
    score_va += clf.score(x[va],y[va])
#     score_te += micro_avg_f1(y_te, clf.predict(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# util.log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
df_stack = pd.DataFrame(index=range(len(df_all)))
for i in range(stack_all.shape[1]):
    df_stack['tfidf_bnb_education_{}'.format(i+1)] = stack_all[:, i]

df_stack.to_csv("drive/My Drive/sougou/data/bnb_prob_education.csv", index=None, encoding='utf8')

2020-04-10 18:17:18 stack:1/5
2020-04-10 18:17:20 va acc:0.594610
2020-04-10 18:17:20 stack:2/5
2020-04-10 18:17:21 va acc:0.590203
2020-04-10 18:17:21 stack:3/5
2020-04-10 18:17:22 va acc:0.587185
2020-04-10 18:17:23 stack:4/5
2020-04-10 18:17:24 va acc:0.593909
2020-04-10 18:17:24 stack:5/5
2020-04-10 18:17:25 va acc:0.590123
2020-04-10 18:17:25 va avg acc:0.591206


In [0]:
from sklearn.naive_bayes import MultinomialNB

############################  Gender ############################
############################ mnb stack ############################
tr_num = 88492
num_class = 2
n = 5

x = x_sp[:tr_num]
y = df_all['Gender'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Gender'][tr_num:]

stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    clf = MultinomialNB()
    clf.fit(x[tr], y[tr])
    y_pred_va = clf.predict_proba(x[va])
    y_pred_te = clf.predict_proba(x_te)
    log('va acc:%f' % clf.score(x[va],y[va]))
#     util.log('te acc:%f' % micro_avg_f1(y_te, clf.predict(x_te)))
    score_va += clf.score(x[va],y[va])
#     score_te += micro_avg_f1(y_te, clf.predict(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# util.log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
df_stack = pd.DataFrame(index=range(len(df_all)))
for i in range(stack_all.shape[1]):
    df_stack['tfidf_mnb_gender_{}'.format(i+1)] = stack_all[:, i]

df_stack.to_csv("drive/My Drive/sougou/data/mnb_prob_gender.csv", index=None, encoding='utf8')

2020-04-10 18:18:44 stack:1/5
2020-04-10 18:18:45 va acc:0.797842
2020-04-10 18:18:45 stack:2/5
2020-04-10 18:18:45 va acc:0.800441
2020-04-10 18:18:45 stack:3/5
2020-04-10 18:18:46 va acc:0.796644
2020-04-10 18:18:46 stack:4/5
2020-04-10 18:18:46 va acc:0.805232
2020-04-10 18:18:46 stack:5/5
2020-04-10 18:18:47 va acc:0.803537
2020-04-10 18:18:47 va avg acc:0.800739


In [0]:
############################  Age ############################
############################ mnb stack ############################
tr_num = 88492
num_class = 6
n = 5

x = x_sp[:tr_num]
y = df_all['Age'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Age'][tr_num:]

stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    clf = MultinomialNB()
    clf.fit(x[tr], y[tr])
    y_pred_va = clf.predict_proba(x[va])
    y_pred_te = clf.predict_proba(x_te)
    log('va acc:%f' % clf.score(x[va],y[va]))
#     util.log('te acc:%f' % micro_avg_f1(y_te, clf.predict(x_te)))
    score_va += clf.score(x[va],y[va])
#     score_te += micro_avg_f1(y_te, clf.predict(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# util.log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
df_stack = pd.DataFrame(index=range(len(df_all)))
for i in range(stack_all.shape[1]):
    df_stack['tfidf_mnb_age_{}'.format(i+1)] = stack_all[:, i]

df_stack.to_csv("drive/My Drive/sougou/data/mnb_prob_age.csv", index=None, encoding='utf8')

2020-04-10 18:19:56 stack:1/5
2020-04-10 18:19:57 va acc:0.479575
2020-04-10 18:19:57 stack:2/5
2020-04-10 18:19:58 va acc:0.481779
2020-04-10 18:19:58 stack:3/5
2020-04-10 18:19:58 va acc:0.474799
2020-04-10 18:19:58 stack:4/5
2020-04-10 18:19:59 va acc:0.478416
2020-04-10 18:19:59 stack:5/5
2020-04-10 18:20:00 va acc:0.475760
2020-04-10 18:20:00 va avg acc:0.478066


In [0]:
############################  Education ############################
############################ mnb stack ############################
tr_num = 88492
num_class = 6
n = 5

x = x_sp[:tr_num]
y = df_all['Education'][:tr_num]
x_te = x_sp[tr_num:]
# y_te = df_all['Education'][tr_num:]

stack = np.zeros((x.shape[0], num_class))
stack_te = np.zeros((x_te.shape[0], num_class))

score_va = 0
# score_te = 0
stratifiedk_fold = StratifiedKFold(n_splits=n, shuffle=True)
for i, (tr, va) in enumerate(stratifiedk_fold.split(x,y)):
    log('stack:%d/%d' % ((i + 1), n))
    clf = MultinomialNB()
    clf.fit(x[tr], y[tr])
    y_pred_va = clf.predict_proba(x[va])
    y_pred_te = clf.predict_proba(x_te)
    log('va acc:%f' % clf.score(x[va],y[va]))
#     util.log('te acc:%f' % micro_avg_f1(y_te, clf.predict(x_te)))
    score_va += clf.score(x[va],y[va])
#     score_te += micro_avg_f1(y_te, clf.predict(x_te))
    stack[va] += y_pred_va
    stack_te += y_pred_te
score_va /= n
# score_te /= n
log('va avg acc:%f' % score_va)
# util.log('te avg acc:%f' % score_te)
stack_te /= n
stack_all = np.vstack([stack, stack_te])
df_stack = pd.DataFrame(index=range(len(df_all)))
for i in range(stack_all.shape[1]):
    df_stack['tfidf_mnb_education_{}'.format(i+1)] = stack_all[:, i]

df_stack.to_csv("drive/My Drive/sougou/data/mnb_prob_education.csv", index=None, encoding='utf8')

2020-04-10 18:20:43 stack:1/5
2020-04-10 18:20:44 va acc:0.518617
2020-04-10 18:20:44 stack:2/5
2020-04-10 18:20:45 va acc:0.515001
2020-04-10 18:20:45 stack:3/5
2020-04-10 18:20:45 va acc:0.519155
2020-04-10 18:20:46 stack:4/5
2020-04-10 18:20:46 va acc:0.515651
2020-04-10 18:20:46 stack:5/5
2020-04-10 18:20:47 va acc:0.512826
2020-04-10 18:20:47 va avg acc:0.516250
